In [ ]:
# -*- coding: utf-8 -*-

'''
图像 字塔的一个应用是图像 合。例如 在图像缝合中 你  将两幅 图叠在一  但是由于 接区域图像像素的不 续性 整幅图的效果看 来会 很差。 时图像 字塔就可以排上用场了 他可以帮你实现无缝 接。  的 一个经典案例就是将两个水果 合成一个 看看下图也 你就明白我在 什么 了。


实现上 效果的步 如下
1.  入两幅图像 苹果和句子
2. 构建苹果和橘子的 斯 字塔 6 层
3. 根据 斯 字塔 算拉普拉斯 字塔
4. 在拉普拉斯的每一层  图像 合 苹果的左 与橘子的右  合  5. 根据 合后的图像 字塔 建原始图像。
'''

import cv2
import numpy as np, sys

A = cv2.imread('apple.jpg')
B = cv2.imread('orange.jpg')

# generate Gaussian pyramid for A
G = A.copy()
gpA = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpA.append(G)

# generate Gaussian pyramid for B
G = B.copy()
gpB = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpB.append(G)

# generate Laplacian Pyramid for A
lpA = [gpA[5]]
for i in range(5, 0, -1):
    GE = cv2.pyrUp(gpA[i])
    L = cv2.subtract(gpA[i - 1], GE)#TODO error
    lpA.append(L)
    #cv2.error: /Users/play/Temp/opencv/modules/core/src/arithm.cpp:659: error: (-209) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function arithm_op


# generate Laplacian Pyramid for B
lpB = [gpB[5]]
for i in range(5, 0, -1):
    GE = cv2.pyrUp(gpB[i])
    L = cv2.subtract(gpB[i - 1], GE)
    lpB.append(L)

# Now add left and right halves of images in each level
# numpy.hstack(tup)
# Take a sequence of arrays and stack them horizontally
# to make a single array.
LS = []
for la, lb in zip(lpA, lpB):
    rows, cols, dpt = la.shape
    ls = np.hstack((la[:, 0:cols / 2], lb[:, cols / 2:]))
    LS.append(ls)

# now reconstruct
ls_ = LS[0]
for i in range(1, 6):
    ls_ = cv2.pyrUp(ls_)
    ls_ = cv2.add(ls_, LS[i])

# image with direct connecting each half
real = np.hstack((A[:, :cols / 2], B[:, cols / 2:]))

cv2.imwrite('Pyramid_blending2.jpg', ls_)
cv2.imwrite('Direct_blending.jpg', real)


In [2]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/12 下午3:06
# @Author  : play4fun
# @File    : 图像金字塔.py
# @Software: PyCharm

"""
图像金字塔.py:
一般情况下 我们 处理是一副具有固定分辨率的图像。
但是有些情况下  我们  对同一图像的不同分 率的子图像  处理。
比如 我们 在一幅图 像中查找某个目标 比如脸 我们不知 目标在图像中的尺寸大小。
 这种情况下 ，我们  创建创建一组图像 ， 这些图像是具有不同分 率的原始图像。
 我们把这组图像叫做图像金字塔
 简单来说 就是同一图像的不同分辨率的子图集合 。

 如果我们把最大的图像放在底部，最小的放在顶部
看起来像一座金字塔， 故而得名：图像金字塔。
  有两类图像金字塔： 高斯金字塔和拉普拉斯金字塔。

高斯金字塔的顶部是通过将底部图像中的连续的行和列去除得到的。
图像中的每个像素值等于下一层图像中 5 个像素的 斯加权平均值。
这样 操作一次一个 MxN 的图像就变成了一个 M/2xN/2 的图像。
所以 这幅图像 的面积就变为原来图像面积的四分之一。 这称为 Octave。
连续进行这样 的操作我们就会得到一个分辨率不断下降的图像金字塔。
我们可以使用函数 cv2.pyrDown() 和 cv2.pyrUp() 构建图像金字塔。


"""

import cv2
import numpy as np

higher_reso = cv2.imread('img/messi5.jpg')
# 函数 cv2.pyrDown() 从一个 分辨率大尺寸的图像向上构建一个金子塔
# 尺寸变小 分辨率降低 。
lower_reso = cv2.pyrDown(higher_reso)
cv2.imshow('lower_reso', lower_reso)

# 从一个低分 率小尺寸的图像向下构建一个 子塔 尺 寸变大 但分 率不会增加
higher_reso2 = cv2.pyrUp(lower_reso)
cv2.imshow('higher_reso2', higher_reso2)

# 你  住的是是 higher_reso2 和 higher_reso 是不同的。因为一旦使 用 cv2.pyrDown() 图像的分辨率就会 低 ,信息就会 丢失

cv2.waitKey(11110)
cv2.destroyAllWindows()


# 什么是轮廓

 轮廓可以简单 为成将连续的点  着 界  在一 的曲线 具有相同
的 色或者灰度。 轮廓在形状分析和物体的检测和 别中很有用。
• 为了更加准确  使用二值化图像。在寻找 轮廓之前 ，要进行阈值化处理
或者 Canny 边界检测。
• 查找 轮廓的函数会修改原始图像。如果你在找到 轮廓之后 想使用原始图
   像的  你应 将原始图像存储到其他变 中。
• 在 OpenCV 中 查找 轮廓就像在黑色背景中超白色物体。你应该记住   找的物体应 是白色而背景应该是黑色。

 我们看看如何在一个二值图像中查找 轮廓 
函数 cv2.findContours() 有三个参数
第一个是 入图像
第二个是轮廓检索模式
第三个是 轮廓近似方法。

返回值有三个
第一个是图像
第二个 是 轮廓
第三个是  轮廓的 层析结构。

轮廓 第二个 回值 是一个 Python 列表
  其中存储 图像中的所有 轮廓。
  每一个 轮廓 是一个 Numpy 数组 包含对 边界点 x y 的坐标。
  
##
在前 的内容中我们使用函数 cv2.findContours 来查找 廓 我们   传入一个参数  廓提取模式 Contour_Retrieval_Mode 。我们总是 把它 置为 cv2.RETR_LIST 或者是 cv2.RETR_TREE 效果 可以

 常我们使用函数 cv2.findContours 在图片中查找一个对 。有时对  可能位于不同的位置。 有些情况 一个形状在另外一个形状的内 。 种 情况下我们称外 的形状为父 内 的形状为子。按照 种方式分类 一幅图 像中的所有 廓之 就建立父子关系。 样我们就可以确定一个 廓与其他  廓是怎样 接的 比如它是不是某个 廓的子 廓 或者是父 廓。 种关系 就成为组织结构

 每一个 廓 包含自己的信息  是父   是子等。OpenCV 使用一个含有四个元素的数组 示。

 [Next Previous  First_Child Parent]。

 如果没有父或子 就为 -1。

 21.5.3  廓检索模式
 RETR_LIST 从  的 度来看  中应是最简单的。它只是提取所有的  廓 而不去创建任何父子关系。换句  就是 人人平等  它们属于同一级组 织 廓。
>>> hierarchy
array([[[ 1, -1, -1, -1],
        [2, 0,-1,-1],
        [3, 1,-1,-1],
        [4, 2,-1,-1],
        [5, 3,-1,-1],
        [6, 4,-1,-1],
        [7, 5,-1,-1],
        [-1, 6, -1, -1]]])

RETR_EXTERNAL 如果你 择 种模式的  只会 回最外 的的 廓  所有的子 廓 会 忽略掉。
>>> hierarchy
array([[[ 1, -1, -1, -1],
        [ 2,  0, -1, -1],
        [-1,  1, -1, -1]]])

RETR_CCOMP 在 种模式下会 回所有的 廓并将 廓分为两级组织结 构。例如 一个对 的外 廓为第 1 级组织结构。而对 内 中空洞的 廓为 第 2 级组织结构 空洞中的任何对 的 廓又是第 1 级组织结构。空洞的组织 结构为第 2 级。
>>> hierarchy
array([[[ 3, -1,  1, -1],
        [ 2, -1, -1,  0],
        [-1,  1, -1,  0],
        [ 5,  0,  4, -1],
        [-1, -1, -1,  3],
        [ 7,  3,  6, -1],
        [-1, -1, -1,  5],
        [ 8,  5, -1, -1],
        [-1,  7, -1, -1]]])

RETR_TREE 终于到最后一个了 也是最完美的一个。 种模式下会 回 所有 廓 并且创建一个完整的组织结构列 。它甚至会告 你 是爷爷 爸 爸 儿子 孙子等。
>>> hierarchy
array([[[ 7, -1,  1, -1],
        [-1, -1,  2,  0],
        [-1, -1,  3,  1],
        [-1, -1,  4,  2],
        [-1, -1,  5,  3],
        [ 6, -1, -1,  4],
        [-1,  5, -1,  4],
        [ 8,  0, -1, -1],
        [-1,  7, -1, -1]]])



In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/12 下午8:38
# @Author  : play4fun
# @File    : 轮廓的性质.py
# @Software: PyCharm

"""
轮廓的性质.py:
"""
import cv2
import numpy as np

# 边界矩形的宽高比
x, y, w, h = cv2.boundingRect(cnt)
aspect_ratio = float(w) / h

# Extent轮廓面积与边界矩形面积的比。
area = cv2.contourArea(cnt)
x, y, w, h = cv2.boundingRect(cnt)
rect_area = w * h
extent = float(area) / rect_area

# Solidity轮廓面积与凸包面积的比。
area = cv2.contourArea(cnt)
hull = cv2.convexHull(cnt)
hull_area = cv2.contourArea(hull)
solidity = float(area) / hull_area

# Equivalent Diameter与轮廓面积相等的圆形的直径
area = cv2.contourArea(cnt)
equi_diameter = np.sqrt(4 * area / np.pi)

# Orientation对象的方向 下 的方法 会返回  长轴和短轴的长度
(x, y), (MA, ma), angle = cv2.fitEllipse(cnt)

# Mask and Pixel Points掩模和像素点
mask = np.zeros(imgray.shape, np.uint8)
cv2.drawContours(mask, [cnt], 0, 255, -1)
pixelpoints = np.transpose(np.nonzero(mask))
# pixelpoints = cv2.findNonZero(mask)
# 第一种方法使用了 Numpy 函数
# 第二种使用 了 OpenCV 函数。
# 结果相同 但还是有点不同。Numpy 给出的坐标是 (row ,colum)
# 而 OpenCV 给出的格式是 (x y )形式的。所以 两个结果基 本是可以互换的。row=x ,colunm=y。

# 最大值和最小值及它们的位置
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(imgray, mask=mask)

# 平均 色及平均灰度 我们也可以使用相同的掩模求一个对 的平均 色或平均灰度
mean_val = cv2.mean(im, mask=mask)

# 极点 一个对象最上面  最下面  最左  最右 的点。
leftmost = tuple(cnt[cnt[:, :, 0].argmin()][0])
rightmost = tuple(cnt[cnt[:, :, 0].argmax()][0])
topmost = tuple(cnt[cnt[:, :, 1].argmin()][0])
bottommost = tuple(cnt[cnt[:, :, 1].argmax()][0])


In [6]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/12 下午7:49
# @Author  : play4fun
# @File    : 21.1.2 怎样绘制轮轮廓.py
# @Software: PyCharm

"""
21.1.2 怎样绘制轮轮廓.py:
函数 cv2.drawContours() 可以 用来绘制 轮廓。它可以根据你提供 的 界点绘制任何形状。
第一个参数是原始图像 
第二个参数是 轮廓 一 个 Python 列表。
第三个参数是 轮廓的索引
在绘制独立 轮廓是很有用 当 设置为 -1 时绘制所有轮廓 。
接下来的参数是 轮廓的颜色和厚度等。
"""

import numpy as np
import cv2

im = cv2.imread('img/cards.png')
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
cv2.imshow('imgray', imgray)

ret, thresh = cv2.threshold(imgray, 244, 255, 0)

img, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print('len(contours', len(contours))
contours2=[cnt for cnt in contours if cv2.contourArea(cnt)>200]#过滤太小的contour
print('过滤太小的contour', len(contours2))

# cv2.drawContours(imgray, contours, -1, (0, 0,255), 3)
cv2.drawContours(im, contours, -1, (255, 0, 0), 3)

if len(contours) > 4:
    # To gdraw an individual contour, say 4th contour:
    # 绘制独立 轮廓 如第四个 轮廓
    cv2.drawContours(image=im, contours=contours, contourIdx=3, color=(0, 0, 255), thickness=3)
    # drawContours(image, contours, contourIdx, color, thickness=None, lineType=None, hierarchy=None, maxLevel=None, offset=None)

    # But most of the time, below method will be useful:
    # 但是大多数时候 下 的方法更有用
    cnt = contours[4]
    cv2.drawContours(im, [cnt], 0, (0, 255, 0), 3)

# 第一个contour
print('contours[0]:', contours[0])
cv2.drawContours(imgray, contours[0], 0, (0, 0, 255), 3)

'''
 这个参数如果 设置为 cv2.CHAIN_APPROX_NONE 
 所有的边界点 都 会被存储。但是我们真的需要那么多点吗 ？
 例如 当我们找的边界是一条直线时。你需要直线上所有的点来表示直线吗 ？
 不是的 我们只需要1 条直线 的两个端点而已。 
 就是 cv2.CHAIN_APPROX_SIMPLE  做的。它会
将 轮廓上的冗余点 去掉， 压缩 轮廓 ，从而节省内存开支。
'''

cv2.imshow('drawContours', im)
cv2.imshow('drawContours-', imgray)
cv2.waitKey(11110)
cv2.destroyAllWindows()


len(contours 118
过滤太小的contour 21
contours[0]: [[[  0   0]]

 [[  0 479]]

 [[639 479]]

 [[639   0]]]


In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/12 下午8:47
# @Author  : play4fun
# @File    : 21.4 轮廓-更多函数.py
# @Software: PyCharm

"""
21.4 轮廓-更多函数.py:
"""

import cv2
import numpy as np

'''
Point Polygon Test
求 图像中的一个点到一个对  廓的最短 离。如果点在 廓的外    回值为 。如果在 廓上  回值为 0。如果在 廓内   回值为正。
下 我们以点 50 50 为例 

dist = cv2.pointPolygonTest(cnt,(50,50),True)

此函数的第三个参数是 measureDist。如果 置为 True 就会 算最 短 离。如果是 False 只会判断 个点与 廓之 的位置关系  回值为 +1 -1 0 

 注意 如果你不  知 具体 离 建 你将第三个参数 为 False  这样速度会提  2 到 3 倍。
'''


In [7]:
# -*- coding: utf-8 -*-
# @Time    : 2017/7/12 下午9:30
# @Author  : play4fun
# @File    : 21.4.3-形状匹配.py
# @Software: PyCharm

"""
21.4.3-形状匹配.py:
函数 cv2.matchShape() 可以帮我们比 两个形状或 廓的相似度。
如果返回值越小， 匹配越好。它是根据 Hu 矩来计算的。文档中对不同的方法有解释。

"""

import cv2
import numpy as np

img1 = cv2.imread('img/star.jpg', 0)
img2 = cv2.imread('img/star2.jpg', 0)

ret, thresh = cv2.threshold(img1, 127, 255, 0)
ret, thresh2 = cv2.threshold(img2, 127, 255, 0)

image,contours, hierarchy = cv2.findContours(thresh, 2, 1)
cnt1 = contours[0]
image,contours, hierarchy = cv2.findContours(thresh2, 2, 1)
cnt2 = contours[0]

ret = cv2.matchShapes(cnt1, cnt2, 1, 0.0)
print(ret)

#Hu 矩是归一化中心矩的线性组合
# 之所以 样做是为了能够获取 代表图像的某个特征的矩函数
# 这些矩函数对某些变化如缩放 旋转，  镜像映射  （ 除了 h1） 具有不变形。

3.2206591324027944


In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import cv2

# im = cv2.imread('test.jpg')#
# im = cv2.imread('poker5hearts.jpg')#
# im = cv2.imread('../data/black-white-rect.png')#contour.jpg #
im = cv2.imread('img/chessboard.jpeg')
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

cv2.imshow("imgray", imgray)

#需要注意的是cv2.findContours()函数接受的参数为二值图，即黑白的（不是灰度图）
# 所以读取的图像要先转成灰度的，再转成二值图
# ret, thresh = cv2.threshold(imgray, 0, 25, 0)
# ret, thresh = cv2.threshold(imgray, 0, 100, 0)
ret, thresh = cv2.threshold(src=imgray, thresh=127, maxval=255, type=cv2.THRESH_BINARY)#src, thresh, maxval, type

cv2.imshow("thresh", thresh)
#轮廓提取模式 Contour_Retrieval_Mode
image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print("contours size: ", len(contours))

img = cv2.drawContours(im, contours, -1, (0,255,0), 3)
# img = cv2.drawContours(im, contours, 3, (255, 0, 0), 3)

cv2.namedWindow("contour.jpg", 0)
cv2.imshow("contour.jpg", img)
cv2.waitKey(11110)
cv2.destroyAllWindows()

In [9]:
# -*-coding:utf8-*-#
__author__ = 'play4fun'
"""
create time:15-10-23 下午12:56
图像的矩可以帮助我们计算图像的质心， 面积等。
"""

import cv2
import numpy as np

from pprint import pprint

img = cv2.imread('img/star.png', 0)
# img = cv2.imread('../data/box.png', 0)

ret, thresh = cv2.threshold(img, 127, 255, 0)
# ret, binary = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
# contours,hierarchy = cv2.findContours(thresh, 1, 2)
image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

print('contours len:', len(contours))

cnt = contours[0]  # 第一个
M = cv2.moments(cnt)

# print(M)
pprint(M)  # 好看一点

# 根据 这些矩的值 我们可以 计算出对象的重心
cx = int(M['m10'] / M['m00'])
cy = int(M['m01'] / M['m00'])
print('重心:', cx, cy)

#
area = cv2.contourArea(cnt)
print('面积:', area)

# 第二参数可以用来指定对象的形状是闭合的 True   是打开的FALSE 一条曲线 。
perimeter = cv2.arcLength(cnt, True)
print('周长:', perimeter)

'''
将轮廓形状近似到另外一种由更少点组成的 廓形状 新 廓的点的数目 由我们 定的准确度来决定。
使用的Douglas-Peucker算法 
为了帮助理解，假设我们 在一幅图像中查找一个矩形 
但是由于图像的 种种原因，我们不能得到一个完美的矩形 而是一个 坏形状 如下图所示 。 

现在你就可以使用这个函数来近似 个形状  了。
 这个函数的第二个参数叫 epsilon 它是从原始 廓到近似轮廓的最大距离。它是一个准确度参数。  
 选择一个好的 epsilon 对于得到满意结果非常重要
'''
epsilon = 0.1*cv2.arcLength(cnt,True)
print('epsilon:',epsilon)

approx = cv2.approxPolyDP(cnt,epsilon,True)
cv2.drawContours(image,[approx],0,(255,0,0),3)
cv2.imshow('approxPolyDP',image)

cv2.waitKey(11110)
cv2.destroyAllWindows()


contours len: 28
{'m00': 24.0,
 'm01': 4340.0,
 'm02': 784869.3333333333,
 'm03': 141949590.0,
 'm10': 1204.3333333333333,
 'm11': 217777.5833333333,
 'm12': 39382989.7,
 'm20': 60483.0,
 'm21': 10936739.166666666,
 'm30': 3039985.1,
 'mu02': 52.66666666662786,
 'mu03': 4.444444477558136,
 'mu11': -6.027777777751908,
 'mu12': -9.43888887871799,
 'mu20': 48.88425925927004,
 'mu21': 1.6211419716819364,
 'mu30': 8.479758229572326,
 'nu02': 0.09143518518511781,
 'nu03': 0.001575031996549189,
 'nu11': -0.010464891975263728,
 'nu12': -0.003344974174144955,
 'nu20': 0.08486850565845493,
 'nu21': 0.0005745038528978878,
 'nu30': 0.0030050753478904303}
重心: 50 180
面积: 24.0
周长: 29.798989295959473
epsilon: 2.9798989295959473
